In [ ]:
import pandas as pd
import pymysql
import requests
import datetime
import time

In [ ]:
# Open database connection
db = pymysql.connect(
host='127.0.0.1', port=3306, user='root', passwd='dish0263', db='stock', charset='utf8')

cursor = db.cursor()

create = """ CREATE TABLE major_capital(
              日期 VARCHAR(20),
              證券代號 VARCHAR(20),
              證券名稱 VARCHAR(20),
              外陸資買進股數_不含外資自營商 VARCHAR(20),
              外陸資賣出股數_不含外資自營商 VARCHAR(20),
              外陸資買賣超股數_不含外資自營商 VARCHAR(20),
              外資自營商買進股數 VARCHAR(20),
              外資自營商賣出股數 VARCHAR(20),
              外資自營商買賣超股數 VARCHAR(20),
              投信買進股數 VARCHAR(20),
              投信賣出股數 VARCHAR(20),
              投信買賣超股數 VARCHAR(20),
              自營商買賣超股數 VARCHAR(20),
              自營商買進股數_自行買賣 VARCHAR(20),
              自營商賣出股數_自行買賣 VARCHAR(20),
              自營商買賣超股數_自行買賣 VARCHAR(20),
              自營商買進股數_避險 VARCHAR(20),
              自營商賣出股數_避險 VARCHAR(20),
              自營商買賣超股數_避險 VARCHAR(20),
              三大法人買賣超股數 VARCHAR(20))
        """

insert = """INSERT INTO major_capital(
              日期,
              證券代號,
              證券名稱,
              外陸資買進股數_不含外資自營商,
              外陸資賣出股數_不含外資自營商,
              外陸資買賣超股數_不含外資自營商,
              外資自營商買進股數,
              外資自營商賣出股數,
              外資自營商買賣超股數,
              投信買進股數,
              投信賣出股數,
              投信買賣超股數,
              自營商買賣超股數,
              自營商買進股數_自行買賣,
              自營商賣出股數_自行買賣,
              自營商買賣超股數_自行買賣,
              自營商買進股數_避險,
              自營商賣出股數_避險,
              自營商買賣超股數_避險,
              三大法人買賣超股數) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
               """

drop = """DROP TABLE major_capital"""

In [ ]:
def daily_capital(date):
    
    TWSE_URL = f'http://www.twse.com.tw/fund/T86?response=json&date={date}&selectType=ALL'
    response = requests.get(TWSE_URL)
    data = response.json()
    
    if data['stat'] == 'OK':
        df = pd.DataFrame.from_dict(data["data"])  
        df.columns = data["fields"]
        df.insert(0,'日期',data['date'])
        return df

In [ ]:
#創建 table
cursor.execute(create)

In [ ]:
#獲取歷史資料

start = '2022-2-16'
today = datetime.date.today()

#yesterday = datetime.date.today() + datetime.timedelta(days=-1)

for i in pd.date_range(start, today)[::-1]:
    print(i.strftime('%Y%m%d'))
    df = daily_capital(i.strftime('%Y%m%d'))
    time.sleep(10)
    for i in range(len(df)):
        x = tuple(df.loc[i])
        cursor.execute(insert,x)

In [ ]:
#儲存變更
db.commit()

In [ ]:
#查詢
select = """select * from major_capital where 證券代號 = '2330'
            """
cursor.execute(select)

results = cursor.fetchall()

In [ ]:
results